In [ ]:
import os
import pandas as pd
import numpy as np
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import calculate_metrics_with_ci as metrics
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import HosmerLemeshow
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_multiple_ROCs,plot_calibration_curves,plot_DCA_curve
import logging
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger().setLevel(logging.WARNING) 
matplotlib.rcParams['pdf.fonttype'] = 42

def calculate_metrics_and_ci(csv_path, dataset_name, model_name):
    data = pd.read_csv(csv_path)
    ytrue = data['target']
    yprob = data['probability_1']
    metric, ci = metrics(np.array(ytrue), np.array(yprob))
    
    result = {
        'Dataset': dataset_name,
        'Model': model_name,
        'Threshold': metric['threshold'],
        'ACC': metric['accuracy'],
        'AUC': metric['auc'],
        'Sensitivity': metric['sensitivity'],
        'Specificity': metric['specificity'],
        'NPV': metric['npv'],
        'PPV': metric['ppv'],
        'F1': metric['f1'],
    }
    ci_result = {
        'Dataset': dataset_name,
        'Model': model_name,
        'ACC': ci['accuracy'],
        'AUC': ci['auc'],
        'Sensitivity': ci['sensitivity'],
        'Specificity': ci['specificity'],
        'NPV': ci['npv'],
        'PPV': ci['ppv'],
        'F1': ci['f1'],
    }
    
    return result, ci_result
# Initialize results DataFrame
results = pd.DataFrame(columns=['Dataset', 'Model', 'Threshold', 'ACC', 'AUC', 'Sensitivity', 'Specificity', 'NPV', 'PPV', 'F1'])
ci_results = pd.DataFrame(columns=['Dataset', 'Model', 'ACC', 'AUC', 'Sensitivity', 'Specificity', 'NPV', 'PPV', 'F1'])
# Define the structure of directories and files
base_dir = r"/Users/y2k/work2025/159xbc/run/DL_vit"
datasets = ['train', 'test']  # List of dataset names
models = ['Vit']  # List of model names
gt_list=[]
prob_list=[]
# Iterate over each model and dataset
for model_name in models:
    for dataset_name in datasets:
        csv_path = os.path.join(base_dir,  f'{dataset_name}_metrics_epoch_1.csv')
        res_csv=pd.read_csv(csv_path)
        gt_list.append(res_csv['target'])
        prob_list.append(res_csv['probability_1'])
        plot_multiple_ROCs([res_csv['target']],[res_csv['probability_1']],models=[dataset_name])
        plt.savefig(f'{dataset_name}_ROC.pdf',dpi=300)
        plot_DCA_curve(res_csv['probability_1'],res_csv['target'],model=f"{dataset_name}-vit")
        plt.savefig(f'{dataset_name}_DCA.pdf',dpi=300)
        #csv_path = os.path.join(base_dir, 'model_name', f'{dataset_name}_metrics_epoch_1.csv')
        if os.path.exists(csv_path):
            result, ci_result = calculate_metrics_and_ci(csv_path, dataset_name, model_name)
            results = pd.concat([results, pd.DataFrame([result])], ignore_index=True)
            ci_results = pd.concat([ci_results, pd.DataFrame([ci_result])], ignore_index=True)
        else:
            print(f"File not found: {csv_path}")
plot_multiple_ROCs(gt_list,prob_list,models=datasets)
plt.savefig('train_test.pdf',dpi=300)
# 查看结果
display(results)
results.to_csv('results.csv', index=False)
display(ci_results)
ci_results.to_csv('ci_results.csv', index=False)

In [ ]:
plot_calibration_curves(gt_list,prob_list,models=datasets,title='')
plt.savefig(f'calibration_curve.pdf',dpi=300)

HosmerLemeshow 检验

In [ ]:
for set ,gt, prob in zip(datasets,gt_list,prob_list):
    hl=HosmerLemeshow(gt,prob)
    print(set,hl)
